In [1]:
import awkward as ak
from coffea import processor
from coffea.nanoevents.methods import candidate
import hist
import pandas as pd
import numpy as np
import pathlib
import shutil 
import os
import matplotlib.pyplot as plt
import mplhep as hep
import warnings
warnings.filterwarnings('ignore')

In [2]:
class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass

    def process(self, events):
        dataset = events.metadata['dataset']
        info = ak.zip(
            {
                "runNum": events.runNum,
                "lumiNum": events.lumiNum,
                "evtNum": events.eventNum,
                "nevt": events.eventCounter,
                "iszb": events.isHLT_HIZeroBias,
                "ismb": events.isHLT_HIMinimumBias,
            }
        )
        elctHMT = ak.zip(
            {
                'bits': events.elctHMT_bits,
                'WireNHits': events.elctHMT_WireNHits,
                'sr': events.elctHMT_sr,
            }
        )
        ca4 = ak.zip(
            {
                'size': events.ca4CSCclusterSize,
                "nStation10": events.ca4CSCclusterNstation10,
            }
        )
        run_cut = (info.runNum == 362321)

        zbflag = info.iszb
        mbflag = info.ismb

        zb = run_cut & zbflag
        mb = run_cut & mbflag

        zb_elctHMT = elctHMT[zb]
        mb_elctHMT = elctHMT[mb]

        zbnevts = ak.num(zb_elctHMT, axis=0)
        mbnevts = ak.num(mb_elctHMT, axis=0)

        h_tot = hist.Hist(
            hist.axis.StrCategory(["zb", "mb"], name="flag"),
            hist.axis.Regular(1, 0, 1, name="# of total events")
        )
        h_tot.fill("zb", np.full(zbnevts, 0))
        h_tot.fill("mb", np.full(mbnevts, 0))

        names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']
        hltrange = np.arange(50, 360, 10)
        
        h_zb_nevt = hist.Hist(
            hist.axis.StrCategory(names, growth=True),
            hist.axis.IntCategory(hltrange),
            hist.axis.Regular(3, 0, 3, name="flags"),
            label="zb_nevt"
        )

        h_mb_nevt = hist.Hist(
            hist.axis.StrCategory(names, growth=True),
            hist.axis.IntCategory(hltrange),
            hist.axis.Regular(3, 0, 3, name="flags"),
            label="mb_nevt"
        )
        

        stationInl1 = {
            "ME11": [8, 9, 24],
            "ME12": [7, 10, 12],
            "ME13": [6, 11, 12],
            "ME21": [5, 12, 12],
            "ME22": [4, 13, 12],
            "ME31": [3, 14, 12],
            "ME32": [2, 15, 12],
            "ME41": [1, 16, 12],
            "ME42": [0, 17, 12],
        }
        
        nStationCut = ak.any(ca4.nStation10 <= 1, axis=1)

        def process_region(station_bits, MEx_hlt_range):
            for MEx_hlt_cut in MEx_hlt_range:
                for key, value in station_bits.items():
                    station_bit1, station_bit2, l1_threshold = value
                    l1sel = ak.any(elctHMT.bits >= 2, axis=1) & ak.any(elctHMT.WireNHits > l1_threshold, axis=1)
                    hltcut = ak.any(ca4.size > MEx_hlt_cut, axis=1)

                    zbsel = zb & l1sel & ((elctHMT.sr == station_bit1) | (elctHMT.sr == station_bit2))
                    mbsel = mb & l1sel & ((elctHMT.sr == station_bit1) | (elctHMT.sr == station_bit2))

                    zbfired = ak.count_nonzero(zbsel)
                    mbfired = ak.count_nonzero(mbsel)
                    
                    hlt_zbfired = ak.count_nonzero(zbsel & hltcut)
                    hlt_mbfired = ak.count_nonzero(mbsel & hltcut)  
                    
                    hlt_zbfired_cut = ak.count_nonzero(zbsel & hltcut & nStationCut)
                    hlt_mbfired_cut = ak.count_nonzero(mbsel & hltcut & nStationCut)                        
                                                  
                    h_zb_nevt.fill(key, MEx_hlt_cut, np.full(zbfired, 0))
                    h_zb_nevt.fill(key, MEx_hlt_cut, np.full(hlt_zbfired, 1))
                    h_zb_nevt.fill(key, MEx_hlt_cut, np.full(hlt_zbfired_cut, 2))

                    h_mb_nevt.fill(key, MEx_hlt_cut, np.full(mbfired, 0))
                    h_mb_nevt.fill(key, MEx_hlt_cut, np.full(hlt_mbfired, 1))
                    h_mb_nevt.fill(key, MEx_hlt_cut, np.full(hlt_mbfired_cut, 2))

        process_region(stationInl1, hltrange)

        return {
            'h_tot': h_tot,
            'h_zb_nevt': h_zb_nevt,
            'h_mb_nevt': h_mb_nevt,
        }

    def postprocess(self, accumulator):
        return accumulator

In [4]:
import time
from coffea.nanoevents import BaseSchema
from glob import glob

tstart = time.time()

futures_run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=12),
    schema=BaseSchema,
    maxchunks=50000,
)

fileset = {
    'hitest_run362321': glob('../tmp/*.root')
}

# fileset = {
#     'single': glob('../tmp/HITestRaw1.root')
# }


output = futures_run(
    fileset,
    treename="simpleCSCshowerFilter/hmt",
    processor_instance=MyProcessor(),
)

elapsed = time.time() - tstart

print(output)

Output()

Output()

{'h_tot': Hist(
  StrCategory(['zb', 'mb'], name='flag'),
  Regular(1, 0, 1, name='# of total events'),
  storage=Double()) # Sum: 156637918.0, 'h_zb_nevt': Hist(
  StrCategory(['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42'], growth=True, label='Axis 0'),
  IntCategory([50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350], label='Axis 1'),
  Regular(3, 0, 3, name='flags'),
  storage=Double()) # Sum: 117870.0, 'h_mb_nevt': Hist(
  StrCategory(['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42'], growth=True, label='Axis 0'),
  IntCategory([50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350], label='Axis 1'),
  Regular(3, 0, 3, name='flags'),
  storage=Double()) # Sum: 98137.0}


In [5]:
import pickle
filename = 'rate_hitest_run362321_hlt_nStationcut.pickle'
# filename = 'test.pickle'
outfile = open(filename, 'wb')
pickle.dump(output, outfile)
outfile.close()

In [ ]:
output['h_tot']['zb', :].values()[0]

In [ ]:
output['h_mb_nevt']['ME11', 14, :].values()[0]

In [ ]:
output['h_mb_nevt']['ME11', 14, :].values()[1]

In [ ]:
output['h_zb_nevt']['ME12', 2, :].values()[0]

In [ ]:
output['h_zb_nevt']['ME12', 2, :].values()[1]

In [ ]:
output['h_nevt']['ME12', 1, :].values()[1]

In [ ]:
output['h_nevt']['ME13', 2, :].values()[1]

In [ ]:
output['h_nevt']['ME21', 3, :].values()[1]

In [ ]:
output['h_nevt']['ME22', 4, :].values()[1]

In [ ]:
output['h_nevt']['ME31', 5, :].values()[1]

In [ ]:
output['h_nevt']['ME32', 6, :].values()[1]

In [ ]:
output['h_nevt']['ME41', 7, :].values()[1]

In [ ]:
output['h_nevt']['ME42', 8, :].values()[1]